# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd 
import numpy as np
from sqlalchemy import create_engine


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
text = nltk.download('punkt')

print(text)

True


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katerynaisaieva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/katerynaisaieva/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/katerynaisaieva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
pip install xgboost

     |████████████████████████████████| 1.3 MB 3.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import statements
# from nltk.tokenize import word_tokenize
# from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
#from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn import metrics 

In [29]:
def load_data():
    engine = create_engine('sqlite:///DesasterResponse.db')
    df = pd.read_sql_table('DesasterResponse',con=engine)
    mask = np.where(df['related']=='2',True, False)
    df.loc[df[mask].index, 'related'] = '1'
    X = df['message']
#     y = df[['related', 'request', 'offer',
#        'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
#         'security', 'military', 'child_alone', 'water', 'food', 'shelter',
#        'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
#        'infrastructure_related', 'transport', 'buildings', 'electricity',
#        'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
#        'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
#         'other_weather', 'direct_report']]
    y = df.iloc[:, 4:]
    categories = list(df.columns[4:])
    
    return X, y, categories
    return X, y 

In [4]:
def load_data(database_filepath):
    engine = create_engine('sqlite:///'+database_filepath)
    df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
    X = df['message'].values
    y = df.drop(['id','message','original','genre'],axis=1).values
    categories = list(df.columns[4:])
    return X, y, categories
X, y, categories = load_data('DisasterResponse.db')

In [51]:
#X,y,categories = load_data() 

In [52]:
X[:3]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name'], dtype=object)

In [53]:
y[:3]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [54]:
categories[:8]

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security']

In [57]:
#y['related'].unique() 

### 2. Write a tokenization function to process your text data

In [35]:
#print(stopwords.words("english")) 

- case normalization (convert to all lowercase)
- punctuation removal
- tokenization, lemmatization, and stop word removal using `nltk`
 

In [5]:
def tokenize(text):
 #   text = text.flatten()
   # normalize case and remove punctuation
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [59]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



In [60]:
message = X[1] 
message 

'Is the Hurricane over or is it not over'

In [61]:
tokenize(message)

['hurricane']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect_tfidf',TfidfVectorizer(tokenizer=tokenize, stop_words='english')),
                     ('clf_multi',MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3), n_jobs=-1))
 
     ])

pipeline2 = Pipeline([('vect_tfidf',TfidfVectorizer(tokenizer=tokenize, stop_words='english')),
                     ('clf_multi', MultiOutputClassifier(RandomForestClassifier()))
 
     ])

pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize))
    , ('tfidf', TfidfTransformer())
    , ('clf', MultiOutputClassifier(RandomForestClassifier()))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

In [8]:
# train classifier
pipeline.fit(X_train, y_train)

/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('vect_tfidf',
                 TfidfVectorizer(stop_words='english',
                                 tokenizer=<function tokenize at 0x7f99b3b25a60>)),
                ('clf_multi',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=3),
                                       n_jobs=-1))])

In [76]:
# train classifier
#pipeline2.fit(X_train, y_train)

In [77]:
# train classifier
#pipeline3.fit(X_train, y_train)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)
y_pred 

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [10]:
y_test

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [11]:
mean_abs_error = metrics.mean_absolute_error(y_test, y_pred)
mean_abs_error 

0.0697665547757095

In [82]:
# evaluate classification model
def evaluate_model(y_test, y_pred, categories):
    clsReport = metrics.classification_report(y_test, y_pred, target_names = categories, zero_division=1)
    print("Classification report:", clsReport)

evaluate_model(y_test, y_pred, categories)


Classification report:                         precision    recall  f1-score   support

               related       0.80      0.96      0.87      4991
               request       0.70      0.16      0.26      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.68      0.08      0.14      2670
          medical_help       0.44      0.01      0.01       535
      medical_products       0.50      0.01      0.02       344
     search_and_rescue       1.00      0.00      0.00       159
              security       0.00      0.00      0.00       116
              military       1.00      0.00      0.00       200
           child_alone       1.00      1.00      1.00         0
                 water       0.71      0.06      0.11       418
                  food       0.72      0.11      0.19       745
               shelter       0.71      0.06      0.12       581
              clothing       0.78      0.07      0.13        98
                

In [12]:
accuracy = (y_pred == y_test).mean()
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)


Accuracy: 0.9302334452242905
Precision: 0.6835546783809624
Recall: 0.2814400389199708


### 6. Improve your model
Use grid search to find better parameters. 

In [73]:
#pipeline.get_params() 

In [8]:
def display_results(cv,y_test, y_pred):
 
    mean_abs_error = metrics.mean_absolute_error(y_test, y_pred)
    accuracy = (y_pred == y_test).mean()
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("Mean absolute Error:\n", mean_abs_error)
    print("\nBest Parameters:", cv.best_params_)

In [ ]:
def load_data(database_filepath):
    engine = create_engine('sqlite:///'+database_filepath)
    df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
    X = df['message'].values
    y = df.drop(['id','message','original','genre'],axis=1).values
    categories = list(df.columns[4:])
    return X, y, categories



def build_model():
    pipeline = Pipeline([('vect_tfidf',TfidfVectorizer(tokenizer=tokenize, stop_words='english')),
                     ('clf_multi',MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3), n_jobs=-1))
 
     ])


    parameters = {
         'verbose': [True,False],
        'clf_multi__estimator__metric':  ['euclidean', 'manhattan', 'minkowski'],
        'clf_multi__estimator__weights':  ['uniform', 'distance'],
        'vect_tfidf__ngram_range': [(1,1)],
        'clf_multi__estimator__n_neighbors': [3,5],
         'clf_multi__estimator__p': [2]
 
      }
 
     # create grid search object
    cv = GridSearchCV(pipeline,param_grid = parameters)

    return cv
 
    

    
def tokenize(text):    
    # normalize case and remove punctuation
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)

    # tokenize text
    tokens = word_tokenize(text)

    # lemmatize and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens
 
def display_results(cv,y_test, y_pred):
 
     mean_abs_error = metrics.mean_absolute_error(y_test, y_pred)
    accuracy = (y_pred == y_test).mean()
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("Mean absolute Error:\n", mean_abs_error)
    print("\nBest Parameters:", cv.best_params_)

def evaluate_model(y_test, y_pred, categories):
    clsReport = metrics.classification_report(y_test, y_pred, target_names = categories, zero_division=1)
    print("Classification report:", clsReport)
        
        
def main():
    X, y, categories = load_data('DisasterResponse.db')
    X_train, X_test, y_train, y_test = train_test_split(X, y)
 
    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # display results
    display_results(model,y_test, y_pred)
#    evaluate_model(y_test, y_pred, categories)

main()

/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[Pipeline] ........ (step 1 of 2) Processing vect_tfidf, total=   7.2s
[Pipeline] ......... (step 2 of 2) Processing clf_multi, total=   3.1s


/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[Pipeline] ........ (step 1 of 2) Processing vect_tfidf, total=   7.8s
[Pipeline] ......... (step 2 of 2) Processing clf_multi, total=   0.6s


/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


...

/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[Pipeline] ........ (step 1 of 2) Processing vect_tfidf, total=   6.9s
[Pipeline] ......... (step 2 of 2) Processing clf_multi, total=   0.5s


/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[Pipeline] ........ (step 1 of 2) Processing vect_tfidf, total=   6.8s
[Pipeline] ......... (step 2 of 2) Processing clf_multi, total=   0.5s


/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packa

In [14]:
#update model with best parameters
def build_model_tuned_KNeighbours():
    pipeline = Pipeline([('vect_tfidf',TfidfVectorizer(tokenizer=tokenize, stop_words='english')),
                     ('clf_multi',MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3), n_jobs=-1))
 
     ])


    parameters = {
#          'verbose': [True],
#         'clf_multi__estimator__metric':  ['euclidean'],
#         'clf_multi__estimator__weights':  [ 'distance'],
#         'vect_tfidf__ngram_range': [(1,1)],
#         'clf_multi__estimator__n_neighbors': [3],
#          'clf_multi__estimator__p': [2]
 
      }
      # create grid search object
    cv = GridSearchCV(pipeline,param_grid = parameters)

    return cv
 
    
def main_KNeighbours():
    X, y, categories = load_data('DisasterResponse.db')
    X_train, X_test, y_train, y_test = train_test_split(X, y)
 
    model = build_model_tuned_KNeighbours()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # display results
    display_results(model,y_test, y_pred)
#    evaluate_model(y_test, y_pred, categories)

main_KNeighbours() 

/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/katerynaisaieva/opt/anaconda3/lib/python3.9/site-packa

Accuracy: 0.9187391923507273
Precision: 0.6971058636167304
Recall: 0.09741812679974621
Mean absolute Error:
 0.08126080764927271

Best Parameters: {}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
def findBestModel(X_train, y_train, X_test, y_test):
    algos = {
        'RandomForestClassifier': {
            'model': MultiOutputClassifier(RandomForestClassifier()),
            'params': {
                'clf__estimator__bootstrap': [True],
                'clf__estimator__min_samples_leaf': [1, 2],
                'clf__estimator__min_samples_split': [2, 4],
                'clf__estimator__n_estimators': [100, 200]
                }
        },

    

        'AdaBoostClassifier': {
            'model': MultiOutputClassifier(AdaBoostClassifier()),
            'params': {
                'clf__estimator__n_estimators': [50, 80],
                'clf__estimator__learning_rate': [0.01, 0.1, 0.5]
            }
        }
    }
    scores = []
    for algo_name, config in algos.items():
        pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', config['model'])
        ])
        
        gs = GridSearchCV(pipeline, config['params'], cv=2, n_jobs=1, verbose=2, return_train_score=False)
        gs.fit(X_train, y_train)
        y_pred = gs.predict(X_test)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_,
            'accuracy': (y_pred == y_test).mean(),
            'precision': precision_score(y_test, y_pred, average='weighted', zero_division=1),
            'recall': recall_score(y_test, y_pred, average='weighted', zero_division=1)
        })
    df = pd.DataFrame(scores,columns=['model', 'best_score', 'best_params', 'accuracy', 'precision', 'recall'])
    return df

In [34]:

def BestModelClassifier(X_train, y_train, X_test, y_test):
    algos = {
            'KNeighborsClassifier': {
                'model': MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3)),
                'params': {
#                      'verbose': [True],
#                     'clf__estimator__metric':  ['euclidean'],
#                     'clf__estimator__weights':  [ 'distance'],
                    'vect_tfidf__ngram_range': [(1,1)],
                    'clf__estimator__n_neighbors': [3],
                     'clf__estimator__p': [2]
                    }
            },

            'RandomForestClassifier': {
                'model': MultiOutputClassifier(RandomForestClassifier()),
                'params': {
                    'clf__estimator__bootstrap': [True],
                    'clf__estimator__min_samples_leaf': [1, 2],
                    'clf__estimator__min_samples_split': [2, 4],
                    'clf__estimator__n_estimators': [100, 200]
                    }
            },

    #         'DecisionTreeClassifier' : {
    #             'model': MultiOutputClassifier(DecisionTreeClassifier()),
    #             'params': {
    #                 'clf__estimator__min_samples_split': [2, 4],
    #                 'clf__estimator__min_samples_leaf': [1, 2]
    #             }
    #         },

            'AdaBoostClassifier': {
                'model': MultiOutputClassifier(AdaBoostClassifier()),
                'params': {
                    'clf__estimator__n_estimators': [50, 80],
                    'clf__estimator__learning_rate': [0.01, 0.1, 0.5]
                }
            }


    #          'XGBoost' : {
    #              'model' : MultiOutputClassifier(XGBClassifier(learning_rate =0.1,
    #                                                              n_estimators=1000,
    #                                                              max_depth=5,
    #                                                              min_child_weight=1,
    #                                                              gamma=0,
    #                                                              subsample=0.8,
    #                                                              colsample_bytree=0.8,
    #                                                              objective= 'binary:logistic',
    #                                                              nthread=4,
    #                                                              scale_pos_weight=1,
    #                                                              seed=27)),
    #              'params' : { 
    #                       'max_depth':[4,5,6],
    #                       'min_child_weight':[4,5,6],
    #                       'gamma':[i/10.0 for i in range(0,5)]
    #              }


        }


    scores = []
    for algo_name, config in algos.items():
        pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', config['model'])
            ])

        gs = GridSearchCV(pipeline, config['params'], cv=1, n_jobs=-1 , verbose=2, return_train_score=False)
        gs.fit(X_train, y_train)
        y_pred = gs.predict(X_test)

        scores.append({
                'model': algo_name,
                'best_score': gs.best_score_,
                'best_params': gs.best_params_,
                'accuracy': (y_pred == y_test).mean(),
                'precision': precision_score(y_test, y_pred, average='weighted', zero_division=1),
                'recall': recall_score(y_test, y_pred, average='weighted', zero_division=1)
            })

    df = pd.DataFrame(scores,columns=['model', 'best_score', 'best_params', 'accuracy', 'precision', 'recall'])
    return df 


In [13]:
df = findBestModel(X_train, y_train, X_test, y_test)
df

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] END clf__estimator__bootstrap=True, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time= 2.5min
[CV] END clf__estimator__bootstrap=True, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100; total time= 2.5min
[CV] END clf__estimator__bootstrap=True, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200; total time= 4.3min
[CV] END clf__estimator__bootstrap=True, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200; total time= 4.4min
[CV] END clf__estimator__bootstrap=True, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=100; total time= 1.9min
[CV] END clf__estimator__bootstrap=True, clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_

,model,best_score,best_params,accuracy,precision,recall
0,RandomForestClassifier,0.260350,"{'clf__estimator__bootstrap': True, 'clf__esti...",0.948530,0.780139,0.534274
1,AdaBoostClassifier,0.259028,"{'clf__estimator__learning_rate': 0.5, 'clf__e...",0.948458,0.759580,0.550231


=> Best result shows : AdaBoostClassifier	

In [15]:
clf_learning_rate = df.loc[1, 'best_params']['clf__estimator__learning_rate']
clf_n_estimators = df.loc[1, 'best_params']['clf__estimator__n_estimators']

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(learning_rate=clf_learning_rate, n_estimators=clf_n_estimators)))
])
      
pipeline.fit(X_train, y_train)



Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd101b24af0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.5,
                                                                    n_estimators=80)))])

### 9. Export your model as a pickle file

In [16]:
model_path = '/Users/katerynaisaieva/Desktop/Nanodegree_Projects/Data_Scientist_Nanodegree/Project2_Disaster_messages_Classification/model'
def save_model(model, model_path):
    pickle.dump(model, open(model_path, 'wb'))

save_model(pipeline, 'DesasterResponse_Model_Classifier.pkl')



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
def main():
    if len(sys.argv) == 3:   #sys. argv is the list of commandline arguments passed to the Python program.
        # argv represents all the items that come along via the command line input
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))

        X, y, categories = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

        print('Building model...')
        model = build_model()

        print('Training model...')
        model.fit(X_train, Y_train)

        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, categories)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, 'classifier.pkl')

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database ' \
              'as the first argument and the filepath of the pickle file to ' \
              'save the model to as the second argument. \n\nExample: python ' \
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()
